<a href="https://colab.research.google.com/github/GIBD/GIBDnews/blob/master/GIBDnews_1_Obtencion_Datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **GIBDnews**
**Aplicación de Técnicas de Minería de Textos a noticias** 

**Parte 1**


---
#**Obtención y Preparación de los Textos**

---

**Fuentes de información**

Para este trabajo se consideró trabajar con tres periódicos digitales nacionales: Clarín, Infobae y La Nación. Se seleccionaron 6 secciones de los diarios: Deportes, Economía, Espectáculos, Política, Sociedad, Tecnología.

---

**Carga de librerías**

---

In [0]:
import pandas as pd
import os
import chardet
import numpy as np
import datetime
from os import path

In [2]:
! git clone https://github.com/GIBD/GIBDnews.git

Cloning into 'GIBDnews'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 88 (delta 1), reused 0 (delta 0), pack-reused 82
Unpacking objects: 100% (88/88), done.


---

**Generación de dataset único**


---

In [0]:
# Constantes

# Solamente es necesario cambiar el directorio base


DIRECTORIO_BASE = r'/content/GIBDnews'     
DIRECTORIO_DATA = path.join(DIRECTORIO_BASE, 'data/raw/2020/04-ABR')
DIRECTORIO_EXPORT = path.join(DIRECTORIO_BASE, 'export')

TEMPLATE_EXPORT = 'news_abril'

MES = 4
DIA_INICIO = 11
DIA_FIN = 15

DIARIOS = ['clarin','infobae','lanacion'] # Metadatos para acceder a los archivos JSON
SECCIONES = ['deportes','economia','sociedad','tecnologia']  #, 'politica'
DIAS = list(range(DIA_INICIO, DIA_FIN + 1))

In [0]:
# Utilidades

def generar_nombre_export(nombre_export):
    formato = "%Y%m%d-%H%M%S"
    fecha_actual = datetime.datetime.now().strftime(formato)
    nombre_archivo = f'{TEMPLATE_EXPORT}_{nombre_export}_{fecha_actual}.csv'
    
    return path.join(DIRECTORIO_EXPORT, nombre_archivo)

def log(mensaje, nivel = 0):
    tabs = ('\t' * nivel) if nivel is not 0 else ''
    print(tabs + mensaje)
    
def crear_directorio_si_no_existe(directorio):
    if not path.exists(directorio):
        os.makedirs(directorio)

In [5]:
crear_directorio_si_no_existe(DIRECTORIO_BASE)
crear_directorio_si_no_existe(DIRECTORIO_EXPORT)

os.chdir(DIRECTORIO_DATA) 

# Inicializa dataframe noticias
noticias = pd.DataFrame()

# Recorre los archivos JSON cargando las noticias en el dataframe noticias
for diario in DIARIOS:
    for seccion in SECCIONES:
        for dia in DIAS:
            # Genera el nombre del JSON a procesar
            nombreJSON = f'{diario}_{seccion}_2020{MES:02d}{dia:02d}.json'
            
            log(f'Procesando archivo {nombreJSON}:')

            # Genera la fecha del JSON actual
            fechaJSON = datetime.datetime(2020, MES, dia)
            
            if not path.exists(nombreJSON):
                log('Archivo no encontrado.', 1)
                continue
            
            # Obtiene metadatos del archivo para acceder al encoding
            with open(nombreJSON, 'rb') as f: 
                result = chardet.detect(f.read())
            
            # Obtiene el dataframe a partir del JSON teniendo en cuenta el encoding
            encoding = 'utf-8' if result['encoding'] != 'Windows-1252' else result['encoding']
            
            # convert_dates en False es porque:
            #    1) los archivos de infobae (ej infobae_economia_20200317) tienen un array en la llave date
            #    2) igualmente la fecha se pisa en el siguiente paso, asi que tampoco importa
            df = pd.read_json(nombreJSON, lines=True, encoding=encoding, convert_dates=False)
                
            # Actualiza la fecha en el dataframe
            df['date'] = fechaJSON
            
            #concatena con los anteriores
            noticias = pd.concat([noticias, df], ignore_index=True, sort=False)
            
            log('Archivo procesado correctamente.', 1)   

Procesando archivo clarin_deportes_20200411.json:
	Archivo procesado correctamente.
Procesando archivo clarin_deportes_20200412.json:
	Archivo procesado correctamente.
Procesando archivo clarin_deportes_20200413.json:
	Archivo procesado correctamente.
Procesando archivo clarin_deportes_20200414.json:
	Archivo no encontrado.
Procesando archivo clarin_deportes_20200415.json:
	Archivo no encontrado.
Procesando archivo clarin_economia_20200411.json:
	Archivo procesado correctamente.
Procesando archivo clarin_economia_20200412.json:
	Archivo procesado correctamente.
Procesando archivo clarin_economia_20200413.json:
	Archivo procesado correctamente.
Procesando archivo clarin_economia_20200414.json:
	Archivo no encontrado.
Procesando archivo clarin_economia_20200415.json:
	Archivo no encontrado.
Procesando archivo clarin_sociedad_20200411.json:
	Archivo procesado correctamente.
Procesando archivo clarin_sociedad_20200412.json:
	Archivo procesado correctamente.
Procesando archivo clarin_socied

In [9]:
noticias
#noticias.body[1]

,news_paper,category,link,date,title,abstract,author,key_words,body
0,clarin,deportes,https://www.clarin.com//deportes/andres-perale...,2020-04-11,El “Pablo Vico” de España: tiene 83 años y pas...,Andrés Perales vive hace casi tres décadas en ...,[],MálagaLiga De EspañaCoronavirusCuarentena,La cuarentena nos obligó a quedarnos en casa y...
1,clarin,deportes,https://www.clarin.com//deportes/adelantan-bar...,2020-04-11,Adelantan que en Barcelona se habrían pagado s...,El 'BarçaGate' sigue sumando capítulos. El dia...,[],FC BarcelonaLionel Messi,Ha sido la temporada de los escándalos para Ba...
2,clarin,deportes,https://www.clarin.com//deportes/jana-pittman-...,2020-04-11,"Jana Pittman, la campeona mundial que lucha en...",La atleta es una de los 47 residentes que fuer...,[],CoronavirusAtletismo,"En Argentina, Paula Pareto es la cara más visi..."
3,clarin,deportes,https://www.clarin.com//deportes/oms-pidio-par...,2020-04-11,Coronavirus: la OMS pidió que paren el fútbol ...,La Organización Mundial de la Salud instó a qu...,[],BielorrusiaCoronavirus,La Organización Mundial de la Salud (OMS) le p...
4,clarin,deportes,https://www.clarin.com//deportes/arana-amuchas...,2020-04-11,"El Araña Amuchástegui, el crack que rechazó un...","Brilló en el River de Veira, pero antes fue un...",Ramón Gómez,River PlateSan LorenzoSelección Argentina De F...,El gran dilema de siempre en Córdoba es el gén...
...,...,...,...,...,...,...,...,...,...
710,lanacion,tecnologia,http://www.lanacion.com.ar//tecnologia/playsta...,2020-04-13,"PlayStation 5: así es DualSense, el control de...",NaN,,,La nueva \r\n\r\nconsola\r\n\r\n de Sony ya es...
711,lanacion,tecnologia,http://www.lanacion.com.ar//tecnologia/faceboo...,2020-04-13,"Facebook presenta Torneos, la plataforma para ...",NaN,Europa Press,,Las competencias de \r\n\r\nvideojuegos\r\n\r\...
712,lanacion,tecnologia,http://www.lanacion.com.ar//tecnologia/la-hist...,2020-04-13,"La historia del programador detrás de Petro, l...",NaN,"[Nathaniel Popper, Ana Vanessa Herrero]",,Justo después de la medianoche de un martes a ...
713,lanacion,tecnologia,http://www.lanacion.com.ar//tecnologia/netflix...,2020-04-13,Netflix: cómo configurar las nuevas herramient...,NaN,Débora Slotnisky,,Netflix está estrenando hoy nuevas herramienta...


In [0]:
# Exporta el dataframe de noticias en formato CSV
export_csv = noticias.to_csv(generar_nombre_export('todas'),index = None, header=True)

#noticias.shape


In [12]:
noticias.shape

(715, 9)

In [13]:
pd.value_counts(noticias['category'])

tecnologia    201
deportes      199
sociedad      186
economia      129
Name: category, dtype: int64

---

**Resumen de cantidad de noticias según diario y sección**

---

In [0]:
# Agrupa las noticias por diario y categoría
a = noticias.groupby(['news_paper', 'category'])
a

In [0]:
# Genera un array numpay solo con la cantidad de noticias
b = np.array(a.aggregate(np.count_nonzero)['link'])
b

array([ 40,  36,  36,  36, 111,  45,  60,  75,  48,  48,  90,  90])

In [0]:
# Agrega al array la sumatoria por secciones
b = np.append(b,[b[0:4] + b[4:8] + b[8:12]])
b

array([ 40,  36,  36,  36, 111,  45,  60,  75,  48,  48,  90,  90, 199,
       129, 186, 201])

In [0]:
# Genera un dataframe con la cantidad de noticias por seccion y diario y los totales
resumen = pd.DataFrame({"Clarin":np.append(b[0:4],[b[0:4].sum()]), 
                        "Infobae":np.append(b[4:8],[b[4:8].sum()]),  
                        "La Nacion":np.append(b[8:12],[b[8:12].sum()]),
                        "TOTAL":np.append(b[12:16],[b[12:16].sum()])},
             index=['Deportes', 'Economía','Sociedad','Tecnologia','TOTAL'])
# Muestra el dataframe
resumen.T

,Deportes,Economía,Sociedad,Tecnologia,TOTAL
Clarin,40,36,36,36,148
Infobae,111,45,60,75,291
La Nacion,48,48,90,90,276
TOTAL,199,129,186,201,715


---

Se exporta la tabla resumen como archivo CSV

---

In [0]:
news_resumen = resumen.to_csv (r'/content/GIBDnews/data/news_resumen.csv', index = True, header=True)